In [1]:
import torch
import argparse
import json
import os
from utils import get_embeddings, run_retrieval, evaluate_retrieval, setup_logging
from data.utils import get_dataset
from models import load_model
import torch.nn.functional as F
import logging

parser = argparse.ArgumentParser()

# Args for datasets
parser.add_argument("--data-dir", type=str, default="./data/", help='Data directory.')
parser.add_argument('--dataset-config', default='./configs/fashion200k.json', help='Dataset config file.')
parser.add_argument("--batch-size", type=int, default=512)
parser.add_argument("--num-workers", type=int, default=4)
# Args for models
parser.add_argument('--model-name', type=str, default='ViT-B-16', help='Model name.')
parser.add_argument('--run-name', type=str, default='ViT-B-16_laion2b_s34b_b88k', help='Run name.')
parser.add_argument("--pretrained", type=str, default='laion2b_s34b_b88k', help='Pretrained name.')
parser.add_argument('--cache-dir', default="/home/jupyter/cache", help='Cache directory for models and datasets.')
parser.add_argument('--device', default='cuda', help='Device to use for inference.')
parser.add_argument("--query-prefix", type=str, default='', help="Query prefix if required (ex. 'description: ')")
# Args for evaluations
parser.add_argument('--Ks', default=[1, 10], nargs='+', help='Ks for metrics.')
parser.add_argument("--overwrite-embeddings", action="store_true", default=False)
parser.add_argument("--overwrite-retrieval", action="store_true", default=False)
parser.add_argument("--output-dir", type=str, default='./results')

_StoreAction(option_strings=['--output-dir'], dest='output_dir', nargs=None, const=None, default='./results', type=<class 'str'>, choices=None, help=None, metavar=None)

In [2]:
import sys
sys.argv = []
args = parser.parse_args()

In [3]:
setup_logging()
# Output directory settings
args.output_dir = os.path.join(args.output_dir, os.path.basename(args.dataset_config).replace('.json',''), args.run_name)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir, exist_ok=True)
else:
    logging.warning(f'Output directory {args.output_dir} exists. Ignore this if it is expected.')
with open(os.path.join(args.output_dir, 'args.json'), 'w') as f:
    json.dump(args.__dict__, f, indent=4)
args.embeddings_path = os.path.join(args.output_dir, "embeddings.pt")

# Read dataset config file
with open(args.dataset_config, 'r') as file:
    args.dataset_config = json.load(file)
logging.info("Dataset: " + args.dataset_config["name"])


2025-01-06,22:58:13 | WARNING | Output directory ./results/fashion200k/ViT-B-16_laion2b_s34b_b88k exists. Ignore this if it is expected.
2025-01-06,22:58:13 | INFO | Dataset: Fashion200k


In [4]:
# Load model
model, preprocess, tokenizer = load_model(args)

# Load documenets and generate embeddings
model = model.to(args.device)


2025-01-06,22:58:15 | INFO | Created a temporary directory at /var/tmp/tmphkaml1mf
2025-01-06,22:58:15 | INFO | Writing /var/tmp/tmphkaml1mf/_remote_module_non_scriptable.py
2025-01-06,22:58:15 | INFO | Loaded ViT-B-16 model config.
2025-01-06,22:58:17 | INFO | Loading pretrained ViT-B-16 weights (laion2b_s34b_b88k).


In [5]:
doc_dataset, item_ID = get_dataset(args, tokenizer, preprocess)
logging.info(f"Number of document rows: {len(doc_dataset):,}")


2025-01-06,22:58:23 | INFO | Loading dataset from huggingface.
D2: <dict object at 0x7fdb8967e140>
T4: <class 'datasets.data_files.DataFilesDict'>
# T4
D2: <dict object at 0x7fdc6c3b3320>
T4: <class 'datasets.data_files.DataFilesList'>
# T4
T4: <class 'datasets.data_files.Url'>
# T4
D2: <dict object at 0x7fdbb67d65f0>
# D2
# D2
# D2
2025-01-06,22:58:24 | INFO | Using custom data configuration Marqo--fashion200k-e452c40783d60867
2025-01-06,22:58:24 | INFO | Overwrite dataset info from restored data version if exists.
2025-01-06,22:58:24 | INFO | Loading Dataset info from /home/jupyter/cache/datasets/Marqo___parquet/Marqo--fashion200k-e452c40783d60867/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7
2025-01-06,22:58:24 | WARNING | Found cached dataset parquet (/home/jupyter/cache/datasets/Marqo___parquet/Marqo--fashion200k-e452c40783d60867/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
2025-01-06,22:58:24 | INFO | Loading Dataset info from /

  0%|          | 0/1 [00:00<?, ?it/s]

T4: <class 'data.utils.Transform'>
# T4
D2: <dict object at 0x7fdba4f585f0>
T4: <class 'open_clip.tokenizer.SimpleTokenizer'>
# T4
D2: <dict object at 0x7fdc6c20f870>
D2: <dict object at 0x7fdbac959370>
# D2
D2: <dict object at 0x7fdba40e0dc0>
# D2
D2: <dict object at 0x7fdb8a391cd0>
# D2
D2: <dict object at 0x7fdba40e0cd0>
# D2
D2: <dict object at 0x7fdb8a391d20>
# D2
D2: <dict object at 0x7fdba40e0c30>
# D2
Re: regex.Regex("<start_of_text>|<end_of_text>|'s|'t|'re|'ve|'m|'ll|'d|[\\p{L}]+|[\\p{N}]|[^\\s\\p{L}\\p{N}]+", flags=regex.I | regex.V0)
F2: <function compile at 0x7fdbaca90200>
# F2
# Re
F2: <function _clean_lower at 0x7fdbaca90a70>
# F2
# D2
T4: <class 'torchvision.transforms.transforms.Compose'>
# T4
D2: <dict object at 0x7fdba40e0fa0>
T4: <class 'torchvision.transforms.transforms.Resize'>
# T4
D2: <dict object at 0x7fdba40e70f0>
T4: <class 'collections.OrderedDict'>
# T4
T1: <class 'set'>
F2: <function _load_type at 0x7fdbc40a1b00>
# F2
# T1
T4: <enum 'InterpolationMode'>
# T

In [6]:
os.path.isfile(args.embeddings_path)

False

In [7]:
if not os.path.isfile(args.embeddings_path) or args.overwrite_embeddings:
    logging.info("Getting embeddings of documents")
    embeddings = get_embeddings(model, doc_dataset, args)
    torch.save(embeddings, args.embeddings_path)
else:
    logging.info("Loading embeddings of documents")
    embeddings = torch.load(args.embeddings_path)

2025-01-06,22:58:30 | INFO | Getting embeddings of documents
100%|██████████| 394/394 [1:07:18<00:00, 10.25s/it]


In [8]:
# Run tasks
for task in args.dataset_config["tasks"]:
    task_dir = os.path.join(args.output_dir, task['name'])
    if not os.path.exists(task_dir):
        os.makedirs(task_dir, exist_ok=True)
    logging.info(f'Task: {json.dumps(task, indent=4)}')

    for query_col in task["query_col"]:
        gt_dir = os.path.join(args.data_dir, args.dataset_config["name"], 'gt_query_doc')
        gt_results_path = os.path.join(gt_dir, f"ground_truth_{query_col}-{'+'.join(task['doc_col'])}.json")
        assert os.path.exists(gt_results_path)

        # Ground-truth query-doc
        logging.info("Loading ground truth")
        with open(gt_results_path, "r") as f:
            gt_results = json.load(f)
            test_queries = list(gt_results.keys()) # randomly sampled queries (up to 2000)
        
        # Running retrieval
        retrieval_path = os.path.join(task_dir, f"retrieved_{query_col}-{'+'.join(task['doc_col'])}.json")
        if os.path.exists(retrieval_path) and not args.overwrite_retrieval:
            logging.info("Loading retrieval")
            with open(retrieval_path, "r") as f:
                retrieval_results = json.load(f)
        else:
            logging.info("Running retrieval")
            if len(task['doc_col'])==1:
                doc_embeddings = embeddings[task['doc_col'][0]].to(args.device)
            else:
                assert ('doc_weights' in task and len(task['doc_weights'])==len(task['doc_col'])), \
                    "Must provide the same number of weights for multi-field documents as the number of multi-fields."
                doc_embeddings = F.normalize(torch.stack([w*embeddings[c] for c, w in zip(task['doc_col'], task['doc_weights'])], dim=1).sum(1), dim=-1).to(args.device)
            retrieval_results = run_retrieval(test_queries, item_ID, doc_embeddings, tokenizer, model, max(args.Ks), args)
            with open(retrieval_path, "w") as f:
                json.dump(retrieval_results, f, indent=4)

        # Evaluation Starts
        logging.info("Evaluation Starts")
        output_results = evaluate_retrieval(gt_results, retrieval_results, args)
        output_json = os.path.join(task_dir, f"result_{query_col}-{'+'.join(task['doc_col'])}.json")
        output_json_dict = json.dumps(output_results, indent=4)
        logging.info(output_json_dict)
        with open(output_json, 'w') as f:
            f.write(output_json_dict)

2025-01-07,00:07:58 | INFO | Task: {
    "name": "text-to-image",
    "query_col": [
        "text"
    ],
    "doc_col": [
        "image"
    ]
}
2025-01-07,00:07:58 | INFO | Loading ground truth
2025-01-07,00:07:58 | INFO | Running retrieval
100%|██████████| 2000/2000 [01:32<00:00, 21.56it/s]
2025-01-07,00:09:31 | INFO | Evaluation Starts
2025-01-07,00:09:31 | INFO | For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
2025-01-07,00:09:31 | INFO | 

2025-01-07,00:09:31 | INFO | NDCG@1: 0.0630
2025-01-07,00:09:31 | INFO | NDCG@10: 0.1365
2025-01-07,00:09:31 | INFO | 

2025-01-07,00:09:31 | INFO | MAP@1: 0.0624
2025-01-07,00:09:31 | INFO | MAP@10: 0.1075
2025-01-07,00:09:31 | INFO | 

2025-01-07,00:09:31 | INFO | Recall@1: 0.0624
2025-01-07,00:09:31 | INFO | Recall@10: 0.2297
2025-01-07,00:09:31 | INFO | 

2025-01-07,00:09:31 | INFO | P@1: 0.0630
2025-01-07,00:09:31 | INFO | P@10: 0.0238
2025-01-07,0